In [ ]:
import sys


In [ ]:
K = [
    0x428A2F98,
    0x71374491,
    0xB5C0FBCF,
    0xE9B5DBA5,
    0x3956C25B,
    0x59F111F1,
    0x923F82A4,
    0xAB1C5ED5,
    0xD807AA98,
    0x12835B01,
    0x243185BE,
    0x550C7DC3,
    0x72BE5D74,
    0x80DEB1FE,
    0x9BDC06A7,
    0xC19BF174,
    0xE49B69C1,
    0xEFBE4786,
    0x0FC19DC6,
    0x240CA1CC,
    0x2DE92C6F,
    0x4A7484AA,
    0x5CB0A9DC,
    0x76F988DA,
    0x983E5152,
    0xA831C66D,
    0xB00327C8,
    0xBF597FC7,
    0xC6E00BF3,
    0xD5A79147,
    0x06CA6351,
    0x14292967,
    0x27B70A85,
    0x2E1B2138,
    0x4D2C6DFC,
    0x53380D13,
    0x650A7354,
    0x766A0ABB,
    0x81C2C92E,
    0x92722C85,
    0xA2BFE8A1,
    0xA81A664B,
    0xC24B8B70,
    0xC76C51A3,
    0xD192E819,
    0xD6990624,
    0xF40E3585,
    0x106AA070,
    0x19A4C116,
    0x1E376C08,
    0x2748774C,
    0x34B0BCB5,
    0x391C0CB3,
    0x4ED8AA4A,
    0x5B9CCA4F,
    0x682E6FF3,
    0x748F82EE,
    0x78A5636F,
    0x84C87814,
    0x8CC70208,
    0x90BEFFFA,
    0xA4506CEB,
    0xBEF9A3F7,
    0xC67178F2,
]


## Variables de Hachage
Définition des valeurs initiales (dérivées des racines carrées des huit premiers nombre premier en hexadécimal).

In [ ]:
H_INIT = [
    0x6A09E667,
    0xBB67AE85,
    0x3C6EF372,
    0xA54FF53A,
    0x510E527F,
    0x9B05688C,
    0x1F83D9AB,
    0x5BE0CD19,
]


## Méthodes Auxiliaires

- `rotr_int(x, n)`: Effectue une rotation circulaire de `x` vers la droite de `n` bits.
- `ch(x, y, z)`: Retourne une valeur basée sur un choix conditionnel entre `y` et `z` selon les bits de `x`.
- `maj(x, y, z)`: Calcule la majorité des bits entre `x`, `y` et `z`.
- `sigma0_int(x)`: Applique une combinaison de rotations circulaires à `x` pour produire une valeur.
- `sigma1_int(x)`: Applique une autre combinaison de rotations circulaires à `x` pour produire une valeur.

In [ ]:
def rotr_int(x, n):
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF


def ch(x, y, z):
    return (x & y) ^ (~x & z)


def maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)


def sigma0_int(x):
    return rotr_int(x, 2) ^ rotr_int(x, 13) ^ rotr_int(x, 22)


def sigma1_int(x):
    return rotr_int(x, 6) ^ rotr_int(x, 11) ^ rotr_int(x, 25)


## Méthode auxilières 2
- rotr: Rotation vers la droite d'une chaine de bits de n positions. Cette méthode est utilisée dans les fonctions sigma de l'algorithme.
- shiftr: Décalage vers la droite d'une chaine de bits de n positions. Cette méthode est utilisée dans les fonctions sigma de l'algorithme.
- xor: ou exclusif entre deux chaines de bits.
- sigma0 et sigma1: Fonctions de hachage utilisées dans l'algorithme SHA-256, combine des opérations de rotation et de décalage.

In [ ]:
def rotr(bits, nb):
    if not bits:
        return ""
    n = int(bits, 2)
    mask = (1 << len(bits)) - 1
    nb = nb % len(bits)
    rotated = (n >> nb) | ((n & ((1 << nb) - 1)) << (len(bits) - nb))
    return format(rotated & mask, f"0{len(bits)}b")


def shiftr(bits, nb):
    if not bits:
        return ""
    return "0" * nb + bits[:-nb] if nb < len(bits) else "0" * len(bits)


def xor(bits1, bits2):
    return "".join("1" if bit1 != bit2 else "0" for bit1, bit2 in zip(bits1, bits2))


def sigma0(bits):
    return xor(xor(rotr(bits, 7), rotr(bits, 18)), shiftr(bits, 3))


def sigma1(bits):
    return xor(xor(rotr(bits, 17), rotr(bits, 19)), shiftr(bits, 10))


In [ ]:
def conversionBinaire(message):
    return "".join(["{0:08b}".format(x) for x in message.encode("utf-8")])


def remplissage(message):
    bits = conversionBinaire(message)
    message_size = format(len(message.encode("utf-8")) * 8, "064b")
    bits += "1"
    bits += "0" * ((512 - 64) - len(bits) % 512)
    bits += message_size
    return bits


def décomposition(bits):
    return [bits[i : i + 32] for i in range(0, len(bits), 32)]


#### Fonction `newMot(bits, t)`

- **Paramètres d'entrée :**
    - `bits` : Une liste de mots de 32 bits (en binaire) représentant les données décomposées et étendues.
    - `t` : Un entier représentant l'indice du mot à générer.

- **Sortie :**
    - Retourne un mot de 32 bits (en binaire) calculé à partir des mots précédents et des fonctions auxiliaires `sigma0` et `sigma1` expliqués précédement.

- **Rôle dans SHA-256 :**
    - Génère un nouveau mot de 32 bits (W[t]) en combinant les résultats des fonctions `sigma0` et `sigma1` appliquées sur des mots spécifiques, ainsi que des additions modulo \(2^{32}\). Ces mots étendus sont nécessaires pour les calculs dans les 64 itérations de l'algorithme SHA-256.

#### Fonction `genererListMot(word)`

- **Paramètres d'entrée :**
    - `word` : Une liste contenant les 16 premiers mots de 32 bits (en binaire) générés à partir du message.

- **Sortie :**
    - Retourne une liste étendue contenant 64 mots de 32 bits (en binaire).

- **Rôle dans SHA-256 :**
    - Étend la liste initiale de 16 mots à 64 mots en utilisant la fonction `newMot`. Ces mots étendus sont utilisés dans les 64 itérations de l'algorithme SHA-256 pour effectuer les calculs nécessaires.


In [ ]:
def newMot(bits, t):
    s1 = int(sigma1(bits[t - 2]), 2)
    w7 = int(bits[t - 7], 2)
    s0 = int(sigma0(bits[t - 15]), 2)
    w16 = int(bits[t - 16], 2)
    result = (s1 + w7 + s0 + w16) % (2**32)
    return format(result, "032b")


def genererListMot(word):
    for i in range(16, 64):
        word.append(newMot(word, i))
    return word


- **Paramètres de la fonction :**
    - `word` : Une liste de 64 mots de 32 bits (en binaire) générés à partir du message. Ces mots sont utilisés pour effectuer les calculs dans chaque itération.
    - `H` : Une liste contenant les valeurs de hachage initiales (ou intermédiaires) utilisées pour démarrer les calculs.

- **Ce que fait la fonction :**
    - **Initialisation :** Une copie des valeurs de hachage `H` est créée dans `S` pour éviter de modifier directement `H` pendant les calculs.
    - **Boucle principale (64 itérations) :**
        - À chaque itération, un mot de 32 bits (`word[i]`) est converti en entier (`w_int`).
        - Deux valeurs temporaires, `T1` et `T2`, sont calculées en utilisant des fonctions auxiliaires (`sigma1_int`, `ch`, `sigma0_int`, `maj`) et des constantes (`K[i]`) définits précédement.
        - Les variables temporaires `S` sont mis à jour en effectuant des décalages et des additions basées sur `T1` et `T2`.
    - **Mise à jour des valeurs de hachage :** Après les 64 itérations, les valeurs de `S` sont ajoutées aux valeurs intermédiaires de `H`.
    - **Retourne :** La liste `H` mise à jour, qui contient les nouvelles valeurs de hachage après le traitement d'un bloc de 512 bits.

In [ ]:
def iterateHash(word, H):
    S = H.copy()
    for i in range(64):
        w_int = int(word[i], 2)
        T1 = (
            S[7] + sigma1_int(S[4]) + ch(S[4], S[5], S[6]) + K[i] + w_int
        ) & 0xFFFFFFFF
        T2 = (sigma0_int(S[0]) + maj(S[0], S[1], S[2])) & 0xFFFFFFFF
        S[7] = S[6]
        S[6] = S[5]
        S[5] = S[4]
        S[4] = (S[3] + T1) & 0xFFFFFFFF
        S[3] = S[2]
        S[2] = S[1]
        S[1] = S[0]
        S[0] = (T1 + T2) & 0xFFFFFFFF
    for i in range(8):
        H[i] = (H[i] + S[i]) & 0xFFFFFFFF
    return H


### Explication de la fonction `sha256(message)`

La fonction `sha256` implémente l'algorithme de hachage SHA-256. Voici une explication étape par étape avec des mots simples :

1. **Initialisation des valeurs de hachage :**
    - On commence par copier les valeurs initiales de hachage (`H_INIT`), qui sont des constantes définies au début de l'algorithme.

2. **Remplissage du message :**
    - Le message est converti en une chaîne binaire, puis "rempli" (padding) pour que sa longueur soit un multiple de 512 bits, comme l'exige SHA-256.

3. **Traitement par blocs de 512 bits :**
    - Le message est divisé en blocs de 512 bits.
    - Chaque bloc est décomposé en mots de 32 bits.
    - Des mots supplémentaires sont générés pour étendre la liste à 64 mots.
    - La fonction `iterateHash` est appelée pour mettre à jour les valeurs de hachage en fonction des mots et des constantes de l'algorithme.

4. **Construction du résultat final :**
    - Une fois tous les blocs traités, les valeurs finales de hachage (contenues dans `H`) sont converties en une chaîne hexadécimale.
    - Cette chaîne représente le hash final du message.

### Résumé
La fonction prend un message en entrée, le transforme en blocs de 512 bits, applique des opérations mathématiques et logiques pour chaque bloc, et retourne un hash unique de 256 bits (64 caractères hexadécimaux). Ce hash est une empreinte numérique du message, utilisée pour garantir son intégrité.

In [ ]:
def sha256(message):
    H = H_INIT.copy()
    bin_str = remplissage(message)

    for i in range(0, len(bin_str), 512):
        block = bin_str[i : i + 512]
        word = genererListMot(décomposition(block))
        H = iterateHash(word, H)

    result = ""
    for h in H:
        result += format(h, "08x")
    return result


In [ ]:
# Exemple d'utilisation
message = "abc"
result = sha256(message)
print(f"SHA-256 de '{message}': {result}")


In [ ]:
# Exemple détaillé avec le message "abc"
message = "abc"
print(f"Message original: '{message}'")
print(f"Longueur en octets: {len(message.encode('utf-8'))}")
print(f"Longueur en bits: {len(message.encode('utf-8')) * 8}")


In [ ]:
# Étape 1: Conversion en binaire
bits_originaux = conversionBinaire(message)
print(f"Conversion binaire: {bits_originaux}")
print(f"Longueur: {len(bits_originaux)} bits")
print(f"Vérification: 'a'={ord('a'):08b}, 'b'={ord('b'):08b}, 'c'={ord('c'):08b}")


In [ ]:
# Étape 2: Remplissage (padding)
bits_remplis = remplissage(message)
print(f"Après remplissage: {bits_remplis}")
print(f"Longueur totale: {len(bits_remplis)} bits")
print(f"Nombre de blocs de 512 bits: {len(bits_remplis) // 512}")

# Vérification du remplissage
message_size = format(len(message.encode("utf-8")) * 8, "064b")
print(f"Taille du message (64 derniers bits): {message_size}")
print(f"Bit '1' ajouté: {bits_remplis[24:25]}")
print(f"Zéros ajoutés: {bits_remplis[25:-64]}")


In [ ]:
# Étape 3: Décomposition en mots de 32 bits
mots_32bits = décomposition(bits_remplis)
print(f"Nombre de mots de 32 bits: {len(mots_32bits)}")
print("Premiers mots:")
for i, mot in enumerate(mots_32bits[:8]):
    print(f"  W[{i:2d}] = {mot} = {int(mot, 2):08x}")


In [ ]:
# Étape 4: Génération des mots étendus (W[16] à W[63])
mots_etendus = genererListMot(mots_32bits[:16].copy())
print("Mots étendus (W[16] à W[19]):")
for i in range(16, 20):
    print(f"  W[{i:2d}] = {mots_etendus[i]} = {int(mots_etendus[i], 2):08x}")

# Détail du calcul de W[16]
print(f"\nCalcul de W[16]:")
w14 = mots_etendus[14]
w15 = mots_etendus[15]
w0 = mots_etendus[0]
w1 = mots_etendus[1]

s1_w14 = sigma1(w14)
w7 = mots_etendus[7]
s0_w0 = sigma0(w0)
w16_calc = mots_etendus[1]

print(f"  σ1(W[14]) = σ1({w14}) = {s1_w14}")
print(f"  W[7] = {w7}")
print(f"  σ0(W[0]) = σ0({w0}) = {s0_w0}")
print(f"  W[1] = {w16_calc}")
print(f"  W[16] = (σ1(W[14]) + W[7] + σ0(W[0]) + W[1]) mod 2^32")
print(f"  W[16] = {int(s1_w14, 2):08x} + {int(w7, 2):08x} + {int(s0_w0, 2):08x} + {int(w16_calc, 2):08x}")
print(f"  W[16] = {int(mots_etendus[16], 2):08x}")


In [ ]:
# Étape 5: Initialisation des valeurs de hash
H = H_INIT.copy()
print("Valeurs initiales H[0] à H[7]:")
for i, h in enumerate(H):
    print(f"  H[{i}] = {h:08x}")

print(f"\nValeurs initiales en binaire:")
for i, h in enumerate(H):
    print(f"  H[{i}] = {format(h, '032b')}")


In [ ]:
# Étape 6: Première itération de la boucle principale (i=0)
print("=== ITÉRATION 0 ===")
S = H.copy()
i = 0
w_int = int(mots_etendus[i], 2)

print(f"W[{i}] = {mots_etendus[i]} = {w_int:08x}")
print(f"K[{i}] = {K[i]:08x}")

# Calcul de T1
sigma1_e = sigma1_int(S[4])
ch_efg = ch(S[4], S[5], S[6])
T1 = (S[7] + sigma1_e + ch_efg + K[i] + w_int) & 0xFFFFFFFF

print(f"\nCalcul de T1:")
print(f"  H[7] = {S[7]:08x}")
print(f"  σ1(H[4]) = σ1({S[4]:08x}) = {sigma1_e:08x}")
print(f"  Ch(H[4], H[5], H[6]) = Ch({S[4]:08x}, {S[5]:08x}, {S[6]:08x}) = {ch_efg:08x}")
print(f"  K[{i}] = {K[i]:08x}")
print(f"  W[{i}] = {w_int:08x}")
print(f"  T1 = {T1:08x}")

# Calcul de T2
sigma0_a = sigma0_int(S[0])
maj_abc = maj(S[0], S[1], S[2])
T2 = (sigma0_a + maj_abc) & 0xFFFFFFFF

print(f"\nCalcul de T2:")
print(f"  σ0(H[0]) = σ0({S[0]:08x}) = {sigma0_a:08x}")
print(f"  Maj(H[0], H[1], H[2]) = Maj({S[0]:08x}, {S[1]:08x}, {S[2]:08x}) = {maj_abc:08x}")
print(f"  T2 = {T2:08x}")

print(f"\nÉtat avant mise à jour:")
for j, s in enumerate(S):
    print(f"  S[{j}] = {s:08x}")

# Mise à jour des registres
S[7] = S[6]
S[6] = S[5]
S[5] = S[4]
S[4] = (S[3] + T1) & 0xFFFFFFFF
S[3] = S[2]
S[2] = S[1]
S[1] = S[0]
S[0] = (T1 + T2) & 0xFFFFFFFF

print(f"\nÉtat après mise à jour:")
for j, s in enumerate(S):
    print(f"  S[{j}] = {s:08x}")


In [ ]:
# Étape 7: Exécution complète de la boucle principale (64 itérations)
print("=== EXÉCUTION COMPLÈTE DE LA BOUCLE PRINCIPALE ===")
H_final = iterateHash(mots_etendus, H.copy())

print("État final après les 64 itérations:")
for i, h in enumerate(H_final):
    print(f"  H[{i}] = {h:08x}")

print(f"\nComparaison avec l'état initial:")
for i in range(8):
    print(f"  H[{i}] initial: {H_INIT[i]:08x} → final: {H_final[i]:08x}")


In [ ]:
# Étape 8: Génération du hash final
resultat_final = ""
for h in H_final:
    resultat_final += format(h, "08x")

print("=== RÉSULTAT FINAL ===")
print(f"Hash SHA-256 de '{message}': {resultat_final}")
print(f"Longueur du hash: {len(resultat_final)} caractères hexadécimaux")
print(f"Longueur en bits: {len(resultat_final) * 4} bits")

# Vérification avec la fonction complète
resultat_complet = sha256(message)
print(f"\nVérification avec la fonction complète: {resultat_complet}")
print(f"Résultats identiques: {resultat_final == resultat_complet}")


In [ ]:
# Test avec d'autres messages
messages_test = ["", "a", "ab", "abc", "abcd", "hello", "Hello World!"]

print("=== TESTS AVEC DIFFÉRENTS MESSAGES ===")
for msg in messages_test:
    hash_result = sha256(msg)
    print(f"SHA-256('{msg}') = {hash_result}")
